In [5]:
import plotly
import seaborn as sns
# from matplotlib import pyplot as plt
import matplotlib.pyplot as plt

%matplotlib inline

#todo: 
csv for each cuisine, 
graphing in this enviro, (run other pyplot examples, see if they work, what do they import?)
logistic regession, 
taking spaces out of middle (replace with underscore)
display notebook on webpage (git hub link)

In [10]:
# import matplotlib.pyplot as plt
# from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import os
import numpy as np

In [21]:
#load clean CSVs 
train_clean_df = pd.read_csv('train_clean_data.csv')
train_clean_df.head()

,id,cuisine,text,category_id
0,10259,greek,"romaine_lettuce,black_olives,grape_tomatoes,ga...",0
1,25693,southern_us,"plain_flour,ground_pepper,salt,tomatoes,ground...",1
2,22213,indian,"water,vegetable_oil,wheat,salt",2
3,13162,indian,"black_pepper,shallots,cornflour,cayenne_pepper...",2
4,42779,spanish,"olive_oil,salt,medium_shrimp,pepper,garlic,cho...",3


Machine Learning:

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer 
 
tfidf_vectorizer=TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', stop_words='english') 
# tfidf_vectorizer=TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english') 
# just send in all your docs here 
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(train_clean_df.text)

# get the first vector out (for the first document) 
first_vector_tfidfvectorizer = tfidf_vectorizer_vectors[0] 


In [23]:
# place tf-idf values in a pandas data frame 
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df = df.sort_values(by=["tfidf"],ascending=False)
df.head(10)

,tfidf
garbanzo_beans,0.417649
seasoning,0.408356
black_olives,0.374864
grape_tomatoes,0.372952
romaine_lettuce,0.362357
feta_cheese_crumbles,0.346146
purple_onion,0.247102
pepper,0.198735
garlic,0.165902
pickled_vegetables,0.000000


In [24]:
features = tfidf_vectorizer.fit_transform(train_clean_df.text).toarray()
labels = train_clean_df.category_id
features.shape

(36716, 3262)

Now, each of 4569 consumer complaint narratives is represented by 12633 features, representing the tf-idf score for different unigrams and bigrams.

In [25]:
# train_clean_df['category_id'] = train_clean_df['cuisine'].factorize()[0]
category_id_df = train_clean_df[['cuisine', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'cuisine']].values)
train_clean_df.head()

,id,cuisine,text,category_id
0,10259,greek,"romaine_lettuce,black_olives,grape_tomatoes,ga...",0
1,25693,southern_us,"plain_flour,ground_pepper,salt,tomatoes,ground...",1
2,22213,indian,"water,vegetable_oil,wheat,salt",2
3,13162,indian,"black_pepper,shallots,cornflour,cayenne_pepper...",2
4,42779,spanish,"olive_oil,salt,medium_shrimp,pepper,garlic,cho...",3


In [64]:
# for cuisine, category_id in sorted(category_to_id.items()):
#     features_chi2 = chi2(features, labels == category_id)
#     indices = np.argsort(features_chi2[0])
#     feature_names = np.array(tfidf_vectorizer.get_feature_names())[indices]
#     unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
#     bigrams = [v for v in feature_names if len(v.split(' ')) == 2]

In [27]:
from sklearn.feature_selection import chi2
import numpy as np
N = 3
for cuisine, category_id in sorted(category_to_id.items()):
    features_chi2 = chi2(features, labels == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf_vectorizer.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}':".format(cuisine))
    print("  . Most correlated:\n. {}".format('\n. '.join(unigrams[-N:])))
#     print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

# 'british':
  . Most correlated:
. golden_syrup
. stilton_cheese
. beef_drippings
# 'cajun_creole':
  . Most correlated:
. creole_seasoning
. andouille_sausage
. cajun_seasoning
# 'chinese':
  . Most correlated:
. sesame_oil
. corn_starch
. soy_sauce
# 'french':
  . Most correlated:
. unsalted_butter
. gruyere_cheese
. large_egg_yolks
# 'greek':
  . Most correlated:
. fresh_dill
. feta_cheese
. feta_cheese_crumbles
# 'indian':
  . Most correlated:
. cumin_seed
. ground_turmeric
. garam_masala
# 'irish':
  . Most correlated:
. guinness_beer
. corned_beef
. irish_whiskey
# 'italian':
  . Most correlated:
. parmesan_cheese
. fresh_basil
. grated_parmesan_cheese
# 'japanese':
  . Most correlated:
. dashi
. sake
. mirin
# 'korean':
  . Most correlated:
. sesame_seeds
. kimchi
. gochujang_base
# 'mexican':
  . Most correlated:
. avocado
. salsa
. corn_tortillas
# 'southern_us':
  . Most correlated:
. grits
. baking_powder
. buttermilk
# 'spanish':
  . Most correlated:
. sherry_vinegar
. saf

Model:

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB()
#     LogisticRegression(random_state=0) #gradient decent wasn't able to converg given these inputs
]
CV = 5 #crossvalidation
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []

NameError: name 'MultinomialNB' is not defined

In [67]:
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\hboll\Anaconda3\envs\pythonadv\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-67-1dda683aad3e>", line 3, in <module>
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  File "C:\Users\hboll\Anaconda3\envs\pythonadv\lib\site-packages\sklearn\utils\validation.py", line 73, in inner_f
    return f(**kwargs)
  File "C:\Users\hboll\Anaconda3\envs\pythonadv\lib\site-packages\sklearn\model_selection\_validation.py", line 406, in cross_val_score
    error_score=error_score)
  File "C:\Users\hboll\Anaconda3\envs\pythonadv\lib\site-packages\sklearn\utils\validation.py", line 73, in inner_f
    return f(**kwargs)
  File "C:\Users\hboll\Anaconda3\envs\pythonadv\lib\site-packages\sklearn\model_selection\_validation.py", line 248, in cross_validate
    for train, test in cv.split(X, y, groups))
  File "C:\User

TypeError: object of type 'NoneType' has no len()

In [ ]:
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

In [ ]:
cv_df.groupby('model_name').accuracy.mean()

In [2]:
# JSON files from Kaggle were saved to github for retrevial
# https://www.kaggle.com/c/whats-cooking/data?select=test.json.zip
train_url = "https://raw.githubusercontent.com/hchuhtala/Recipe-ML-App/f22de810927f2ed0c079c1b021cd757a056ef24e/data/train.json"
test_url = "https://raw.githubusercontent.com/hchuhtala/Recipe-ML-App/f22de810927f2ed0c079c1b021cd757a056ef24e/data/test.json"

In [3]:
#Store data in Pandas Data Frames
train_df = pd.read_json(train_url).set_index('id')
train_df.head(10)

,cuisine,ingredients
id,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
22213,indian,"[water, vegetable oil, wheat, salt]"
13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."
6602,jamaican,"[plain flour, sugar, butter, eggs, fresh ginge..."
42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli..."
3735,italian,"[sugar, pistachio nuts, white almond bark, flo..."
16903,mexican,"[olive oil, purple onion, fresh pineapple, por..."


In [4]:
test_df = pd.read_json(test_url).set_index('id')

Data Clean Up:

In [5]:
original_cuisine = train_df["cuisine"].unique()
original_cuisine_list = original_cuisine.tolist()
original_cuisine_list

['greek',
 'southern_us',
 'filipino',
 'indian',
 'jamaican',
 'spanish',
 'italian',
 'mexican',
 'chinese',
 'british',
 'thai',
 'vietnamese',
 'cajun_creole',
 'brazilian',
 'french',
 'japanese',
 'irish',
 'korean',
 'moroccan',
 'russian']

In [6]:
print("There are " + str(train_df.count()[0]) + " records in the raw data set")

There are 39774 records in the raw data set


In [7]:
# #dropping cuisines types that are not in both this data set and API used later in the app
cuisine_list = ['british', 'chinese', 'cajun_creole', 'french', 'greek', 'indian', 'irish', 'italian', 'japanese', 'korean', 'mexican', 'southern_us', 'spanish', 'thai', 'vietnamese']
train_clean_df = train_df[train_df.cuisine.isin(cuisine_list)]
train_clean_df.head()

,cuisine,ingredients
id,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
22213,indian,"[water, vegetable oil, wheat, salt]"
13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."
42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli..."


In [8]:
cuisine_list = train_clean_df["cuisine"].unique()
cuisine_list = cuisine_list.tolist()
cuisine_list

['greek',
 'southern_us',
 'indian',
 'spanish',
 'italian',
 'mexican',
 'chinese',
 'british',
 'thai',
 'vietnamese',
 'cajun_creole',
 'french',
 'japanese',
 'irish',
 'korean']

In [9]:
# cuisine_df = pd.DataFrame(cuisine_list)
# cuisine_df

,0
0,greek
1,southern_us
2,indian
3,spanish
4,italian
5,mexican
6,chinese
7,british
8,thai
9,vietnamese


In [10]:
print("We will be working with " + str(train_clean_df.count()[0]) + " records for our machine learning")

We will be working with 36716 records for our machine learning


In [11]:
type(train_clean_df.ingredients)

pandas.core.series.Series

In [12]:
train_clean_df['text'] = train_clean_df['ingredients'].apply(lambda x:','.join([str(i) for i in x]))
train_clean_df

C:\Users\hboll\Anaconda3\envs\pythonadv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,cuisine,ingredients,text
id,,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes...","romaine lettuce,black olives,grape tomatoes,ga..."
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...","plain flour,ground pepper,salt,tomatoes,ground..."
22213,indian,"[water, vegetable oil, wheat, salt]","water,vegetable oil,wheat,salt"
13162,indian,"[black pepper, shallots, cornflour, cayenne pe...","black pepper,shallots,cornflour,cayenne pepper..."
42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli...","olive oil,salt,medium shrimp,pepper,garlic,cho..."
...,...,...,...
29109,irish,"[light brown sugar, granulated sugar, butter, ...","light brown sugar,granulated sugar,butter,warm..."
11462,italian,"[KRAFT Zesty Italian Dressing, purple onion, b...","KRAFT Zesty Italian Dressing,purple onion,broc..."
2238,irish,"[eggs, citrus fruit, raisins, sourdough starte...","eggs,citrus fruit,raisins,sourdough starter,fl..."


In [13]:
# codes, uniques = pd.factorize(train_clean_df.cuisine)
# codes

array([ 0,  1,  2, ..., 13,  6,  5], dtype=int64)

In [14]:
# train_clean_df['cat_id'] = codes
# train_clean_df.head()

C:\Users\hboll\Anaconda3\envs\pythonadv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,cuisine,ingredients,text,cat_id
id,,,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes...","romaine lettuce,black olives,grape tomatoes,ga...",0
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...","plain flour,ground pepper,salt,tomatoes,ground...",1
22213,indian,"[water, vegetable oil, wheat, salt]","water,vegetable oil,wheat,salt",2
13162,indian,"[black pepper, shallots, cornflour, cayenne pe...","black pepper,shallots,cornflour,cayenne pepper...",2
42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli...","olive oil,salt,medium shrimp,pepper,garlic,cho...",3
